In [1]:
import numpy as np 
import torchvision.datasets as datasets

Prepare dataset by flattening out the image. 

In [2]:
train_set = datasets.MNIST('./data', train=True, download=True)
test_set = datasets.MNIST('./data', train=False, download=True)

# Check the lengths of train sets and test sets
assert len(train_set) == 60000 and len(test_set) == 10000

X_train = np.array([picture.numpy().reshape(-1) for picture in train_set.data]).T / 255.
Y_train = train_set.targets.numpy() 
X_test = np.array([picture.numpy().reshape(-1) for picture in test_set.data]).T / 255.
Y_test = test_set.targets.numpy()

# Check shapes 
assert X_train.shape == (784, 60000) and Y_train.shape == (60000, )


Define the basic functions. 

In [9]:
def initialize_params(): 
    W1 = np.random.uniform(-1, 1, size=(10, 784)) 
    b1 = np.random.uniform(-1, 1, size=(10, 1)) 
    W2 = np.random.uniform(-1, 1, size=(10, 10)) 
    b2 = np.random.uniform(-1, 1, size=(10, 1)) 
    return W1, b1, W2, b2 

def oneHot(Y): 
    # Y is 60000 
    oneHotY = np.zeros((10, Y.size))
    oneHotY[Y, np.arange(Y.size)] = 1 
    return oneHotY # 10x60000

def ReLU(Z): 
    return np.maximum(0, Z)

def ReLU_d(Z): 
    return Z > 0

def softMax(X:np.array): 
    x_max = np.max(X, axis=0)
    X = X - x_max
    return np.exp(X) / np.sum(np.exp(X), axis=0) 

def softMax_d(X:np.array): 
    sm = softMax(X)
    return - (np.diag(sm.sum(axis=1)) - np.matmul(sm, np.transpose(sm))) / X.shape[1]

In [10]:
k = np.arange(9).reshape(3, 3) 
np.diag(k)

array([0, 4, 8])

Now define forward prop and backprop. 

In [14]:
def forwardProp(W1, b1, W2, b2, X): 
    # Z1 12x1, W1 12x784 , X 784x1, b1 12x1
    Z1 = np.matmul(W1, X) + b1
    # A1 12x60000
    A1 = ReLU(Z1) 
    
    # Z2 10x1, W2 10x12, A1 12x60000, b2 10x60000
    Z2 = np.matmul(W2, A1) + b2 
    # A2 10x60000
    A2 = softMax(Z2) 
    return Z1, A1, Z2, A2 

def backProp(Z1, A1, Z2, A2, W1, W2, X, Y): 
    N = Y.size
    oneHotY = oneHot(Y)
    
    # 10x1 = 10x10 10x1
    error2 = A2 - oneHotY
    # error2 = np.matmul(np.transpose(softMax_d(Z2)), A2 - oneHotY) 
    # 10x12 = 10x1 1x12
    dW2 = 1/N * np.matmul(error2, A1.T)
    # 10x1
    dB2 = 1/N * error2.sum(axis=1)
    
    # 12x1 = 12x1 .* 12x10 10x1
    error1 = np.vectorize(ReLU_d)(Z1) * np.matmul(W2.T, error2) 
    # 12x784 = 12x1 1x784
    dW1 = 1/N * np.matmul(error1, X.T)
    # 12x1 
    dB1 = 1/N * error1.sum(axis=1)

    return dW1, dB1, dW2, dB2 

Define gradient descent. 

In [15]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1.reshape(-1, 1)    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2.reshape(-1, 1)       
    return W1, b1, W2, b2

def gradient_descent(X, Y, alpha, iterations): 
    W1, b1, W2, b2 = initialize_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forwardProp(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backProp(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [16]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.1, 500)

Iteration:  0
[4 7 4 ... 4 7 0] [5 0 4 ... 5 6 8]
0.10175
Iteration:  10
[8 0 4 ... 4 6 0] [5 0 4 ... 5 6 8]
0.20715
Iteration:  20
[8 0 4 ... 4 6 0] [5 0 4 ... 5 6 8]
0.2333
Iteration:  30
[9 0 4 ... 4 6 0] [5 0 4 ... 5 6 8]
0.2579166666666667
Iteration:  40
[9 0 4 ... 4 6 0] [5 0 4 ... 5 6 8]
0.2836166666666667
Iteration:  50
[9 0 4 ... 4 6 0] [5 0 4 ... 5 6 8]
0.2981333333333333
Iteration:  60
[9 0 4 ... 4 6 0] [5 0 4 ... 5 6 8]
0.32313333333333333
Iteration:  70
[2 0 4 ... 4 6 0] [5 0 4 ... 5 6 8]
0.3860166666666667
Iteration:  80
[2 0 4 ... 4 6 0] [5 0 4 ... 5 6 8]
0.43856666666666666
Iteration:  90
[2 0 4 ... 4 6 0] [5 0 4 ... 5 6 8]
0.48101666666666665
Iteration:  100
[2 0 9 ... 4 6 0] [5 0 4 ... 5 6 8]
0.51125
Iteration:  110
[2 0 9 ... 4 6 0] [5 0 4 ... 5 6 8]
0.53455
Iteration:  120
[2 0 9 ... 4 6 0] [5 0 4 ... 5 6 8]
0.5567833333333333
Iteration:  130
[2 0 9 ... 5 6 0] [5 0 4 ... 5 6 8]
0.5744
Iteration:  140
[2 0 9 ... 5 6 0] [5 0 4 ... 5 6 8]
0.5921
Iteration:  150
[2 0 9 